In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 14228



### data

In [2]:
sas.submitLST(f"""
    data cars2;
        set sashelp.cars (obs=3);
        if strip(model) in ('TSX 4dr', 'TL 4dr', 'MDX') then do;
            msrp = .;
            invoice = .;
            mpg_city = mpg_city + 1.2;
            mpg_highway = .;
        end;
        if strip(model) in ('3.5 RL 4dr') then do;
            type = '';
            origin = 'abc';
        end;
        keep model type origin msrp invoice mpg_city mpg_highway;
    run;
              
    proc sort data=cars2 nodupkey;
        by model;
    run;

    proc print data=cars2 (obs=5);
    run;
""")

Obs,Model,Type,Origin,MSRP,Invoice,MPG_City,MPG_Highway
1,MDX,SUV,Asia,.,.,18.2,.
2,RSX Type S 2dr,Sedan,Asia,"$23,820","$21,761",24.0,31
3,TSX 4dr,Sedan,Asia,.,.,23.2,.


In [3]:
sas.submitLST(
f"""
data cars;
    set sashelp.cars (obs=3);
    keep model type origin msrp invoice mpg_city mpg_highway;
run;

proc print data=cars (obs=5);
run;
""")

Obs,Model,Type,Origin,MSRP,Invoice,MPG_City,MPG_Highway
1,MDX,SUV,Asia,"$36,945","$33,337",17,23
2,RSX Type S 2dr,Sedan,Asia,"$23,820","$21,761",24,31
3,TSX 4dr,Sedan,Asia,"$26,990","$24,647",22,29


In [9]:
sas.submitLST(
f"""
title;
""", method="listorlog")

25                                                         The SAS System                       Wednesday, July 24, 2024 01:51:00 PM

200        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
200      !  graphics on / outputfmt=png;
201        
202        
203        title;
204        
205        
206        
207        ods html5 (id=saspy_internal) close;ods listing;
208        
26                                                         The SAS System                       Wednesday, July 24, 2024 01:51:00 PM

209        


### basic

In [10]:
# order is compared
sas.submitLST(
f"""
proc sort data=cars nodupkey out=_a;
    by descending model;

proc sort data=cars2 nodupkey out=_b;
    by model;
""", method="listonly")

sas.submitLST(
f"""
proc compare base=_a compare=_b;
run;
""", method="listorlog")

print(f"{sas.saslog().count('ERROR') = }")

sas.saslog().count('ERROR') = 0


In [5]:
# use id to match comparison records
sas.submitLST(
f"""
proc sort data=cars nodupkey out=_a;
    by model;

proc sort data=cars2 nodupkey out=_b;
    by model;

proc compare base=_a compare=_b;
    id model;
run;
""")

### accuracy

In [22]:
sas.submitLST(
f"""
data a;
    input x;
    datalines;
1.23456
1.23456
;
run;

data b;
    input x;
    datalines;
1.23457
1.23456
;
run;

title;
proc print data=a;
run;
proc print data=b;
run;
""", method="listorlog")

Obs,x
1,1.23456
2,1.23456
Obs,x
1,1.23457
2,1.23456


In [26]:
sas.submitLST(
f"""
proc compare base=a compare=b;
run;
""", method="listorlog")

In [28]:
sas.submitLST(
f"""
proc compare base=a compare=b maxprint=0;
run;
""", method="listorlog")

In [27]:
sas.submitLST(
f"""
proc compare base=a compare=b method=relative;
run;
""", method="listorlog")